In [ ]:
from google.colab import files
uploaded=files.upload()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import metrics
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

#Lectura de los datos
data = pd.read_csv("user_fechas.csv")

#Selección de los alumnos desados y las columnas deseadas
columnas = list(data.columns.values)
columnas.remove('User')
data['Descartar'] = data[columnas].sum(axis=1).copy()
data_nota = data[(data['Descartar']>1) & (data['Nota']>0.0)]
print('Longitud inicial con todos los estudiantes', len(data))
print('Longitud final con los estudiantes que al menos tienen un evento', len(data_nota))

"""
X = data[['video_video','video_foro', 'video_foro_search', 'video_foro_response', 'video_foro_comment', 'video_foro_created', 'video_problem',
       'problem_problem','problem_video','problem_foro',  'problem_foro_search', 'problem_foro_response', 'problem_foro_comment','problem_foro_created',
       'foro_foro','foro_video', 'foro_search_video', 'foro_response_video', 'foro_comment_video','foro_created_video','foro_problem',
       'foro_search_problem', 'foro_response_problem', 'foro_comment_problem', 'foro_created_problem',
       'foro_comment_foro_comment','foro_comment_foro_search' ,'foro_comment_foro_response', 'foro_comment_foro_created', 'foro_search_foro_comment' , 'foro_search_foro_search' , 'foro_search_foro_response',  'foro_search_foro_created' , 'foro_response_foro_comment',  'foro_response_foro_search' , 'foro_response_foro_response',  'foro_response_foro_created' , 'foro_created_foro_comment' , 'foro_created_foro_search' , 'foro_created_foro_response' ,'foro_created_foro_created']]
X = data[['video_video','video_foro', 'video_problem',
       'problem_problem','problem_video','problem_foro',
       'foro_foro','foro_video', 'foro_problem'
       ]]

y = data['Nota']
"""

#Eleccion de las columnas a evaluar
X = data_nota[['video_video','video_foro', 'video_problem', 'problem_problem','problem_video','problem_foro', 'foro_foro','foro_video', 'foro_problem']].copy()
Y = data_nota['Nota'].copy()

#Separar los datos en entrenamiento y prueba para probar los algoritmos
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)


Se va a pasar a utlizar diferentes tipos de regresiones. Se van a seguir estos pasos:


*   Se define el algoritmo a utilizar
*   Se entrena/aplica el modelo
*   Se realiza la predicción
*   Se imprimen las predicciones, el score, los coeficientes y el error



#RANDOM FOREST

In [ ]:
#Se define el algoritmo a utilizar
parameters = [{'n_estimators' : [10, 20, 50, 100, 200, 500, 1000], 'max_leaf_nodes' : [5, 10, 20, 50]}]
regressor = RandomForestRegressor()
rf = GridSearchCV(regressor,parameters)

#Entreno/Aplicar el modelo
rf.fit(X_train, y_train)

#Realizo una predicción
y_predic_rf = rf.best_estimator_.predict(X_test)

In [ ]:
df_rf = pd.DataFrame({'Actual': y_test, 'Predicted': y_predic_rf})
df_rf_sort = df_rf.head(1000).sort_values(['Predicted'],ascending=False)
df_rf_sort

In [ ]:
rf.best_estimator_.feature_importances_

In [ ]:
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_predic_rf)))


In [ ]:
print("Coeficientes/pendientes del mejor estimador: ", rf.best_estimator_)

In [ ]:
#Feautures importnce dividir entre 10 nota/10 = nota/10

#REGRESIÓN RIDGE

In [ ]:
#Defino el algoritmo a utilizar
params = [{'alpha' : [0.0000000001,0.00001,0.0001,0.001,0.01,0.1,1,10,100]}] #Ajustar los parametros
regressor = Ridge(positive=True) #Para que no haya pesos negativos
grid = GridSearchCV(regressor,params)

#Entreno/Aplicar el modelo
grid.fit(X_train, y_train)

#Realizo una predicción
y_predic_grid = grid.best_estimator_.predict(X_test)

In [ ]:
df_grid = pd.DataFrame({'Actual': y_test, 'Predicted': y_predic_grid})
df_grid_sort = df_grid.head(1000).sort_values(['Predicted'],ascending=False)
df_grid_sort

In [ ]:
print('MODELO REGRESIÓN RIDGE')
score = grid.score(X_train,y_train)
print("El score es:", score)
print("Coeficientes/pendientes del mejor estimador: ", grid.best_estimator_.coef_)
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_predic_grid)))
print("Coeficientes/pendientes del mejor estimador: ", grid.best_estimator_)

#REGRESIÓN LINEAL MÚLTIPLE

In [ ]:
from sklearn import datasets, linear_model
lr_multiple = linear_model.LinearRegression()
lr_multiple.fit(X_train, y_train)
y_pred_multiple = lr_multiple.predict(X_test)

df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred_multiple})
df1 = df.head(1000).sort_values(['Predicted'],ascending=False)
df1

#Defino el algoritmo a utilizar
lr_multiple = linear_model.LinearRegression()
#Entreno/Aplicar el modelo
lr_multiple.fit(X_train, y_train)

#Realizo una predicción
y_predic_multiple = lr_multiple.predict(X_test)


In [ ]:
df_multiple = pd.DataFrame({'Actual': y_test, 'Predicted': y_predic_multiple})
df_multiple_sort= df.head(1000).sort_values(['Predicted'],ascending=False)
df_multiple_sort

In [ ]:
print('MODELO REGRESIÓN LINEAL MULTIPLE')
score = lr_multiple.score(X_train, y_train)
print("El score es:", score)
print("Coeficientes/pendientes del mejor estimador: ", lr_multiple.coef_)
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_predic_multiple)))

#KNN

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
#Defino el algoritmo a utilizar
params = [{ 'n_neighbors' : [1, 3, 5, 7, 10, 15, 20, 40, 80], 'weights' : ['uniform','distance']}] #Ajustar los parametros
model = KNeighborsRegressor()
knn = GridSearchCV(model,params)

#Entreno/Aplicar el modelo
knn.fit(X_train, y_train)

#Realizo una predicción
y_predic_knn = knn.best_estimator_.predict(X_test)


In [ ]:
df_knn = pd.DataFrame({'Actual': y_test, 'Predicted': y_predic_knn})
df_knn_sort= df.head(1000).sort_values(['Predicted'],ascending=False)
df_knn_sort

In [ ]:
print('MODELO REGRESIÓN LINEAL MULTIPLE')
score = knn.score(X_train, y_train)
print("El score es:", score)
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_predic_knn)))
print("Coeficientes/pendientes del mejor estimador: ", knn.best_estimator_)

In [ ]:
#LECTURA Y SELECCIÓN DE LOS DATOS
data = pd.read_csv("user_fechas.csv")
columnas = list(data.columns.values)
data['Descartar'] = data[columnas].sum(axis=1).copy()
data_nota = data[(data['Descartar']>1) & (data['Nota']>0.0)]

#OBTENCIÓN DE X_TRAIN, X_TEST, Y_TRAIN E Y_TEST
#Eleccion de las columnas a evaluar
X = data_nota[['video_video','video_foro', 'video_problem', 'problem_problem','problem_video','problem_foro', 'foro_foro','foro_video', 'foro_problem']].copy()
Y = data_nota['Nota'].copy()

#Separar los datos en entrenamiento y prueba para probar los algoritmos
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

#SE DEFINE EL ALGORITMO A UTILIZAR
#Ajustar los parametros
params = [{ 'n_neighbors' : [1, 3, 5, 7, 10, 15, 20, 40, 80], 'weights' : ['uniform','distance']}]
       #
       # [{'alpha' : [0.0000000001,0.00001,0.0001,0.001,0.01,0.1,1,10,100]}]
       # [{'n_estimators' : [10, 20, 50, 100, 200, 500, 1000], 'max_leaf_nodes' : [5, 10, 20, 50]}]

#Elegir el modelo
model = KNeighborsRegressor()
      # LinearRegression()
      # Ridge()
      # RandomForestRegressor()


algoritmo = GridSearchCV(model,params) #CREAR EL MODELO
algoritmo.fit(X_train, y_train) #SE ENTREA EL MODELO
y_predic= knn.best_estimator_.predict(X_test) #SE REALIZA UNA PREDICCIÓN
mse = np.sqrt(metrics.mean_squared_error(y_test, y_predic)) #ERROR



